<a href="https://colab.research.google.com/github/firdausjuanda/vortue/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 11.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# load dataset
df = pd.read_csv('data.csv', delimiter=',', encoding='latin-1')
df = df[['permit_category', 'permit_description']]
df = df.dropna()

# tokenize dataset
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['permit_description'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# pad sequences
max_length = 200
sequences = tokenizer.texts_to_sequences(df['permit_description'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# convert permit category to numerical labels
labels = df['permit_category'].unique()
labels_dict = dict(zip(labels, np.arange(len(labels))))
df['LABEL'] = df['permit_category'].map(labels_dict)

# split dataset into train and test sets
train_size = int(0.8 * len(df))
train_data = padded_sequences[:train_size]
train_labels = df['LABEL'][:train_size].values
test_data = padded_sequences[train_size:]
test_labels = df['LABEL'][train_size:].values

# build RNN model
model = Sequential()
model.add(Embedding(5000, 32, input_length=max_length))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))

# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# train model
num_epochs = 10
history = model.fit(train_data, train_labels, epochs=num_epochs, validation_data=(test_data, test_labels), verbose=2)

# evaluate model
loss, accuracy = model.evaluate(test_data, test_labels, verbose=0)
print('Test accuracy: %f' % (accuracy*100))

# save model
model.save('permit_category_model_rnn.h5')


Found 2755 unique tokens.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 200, 32)           8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 8)                 520       
                                                                 
Total params: 179,272
Traina